In [1]:
from google.colab import files
uploaded = files.upload()

Saving test-data.dat to test-data.dat
Saving test-label.dat to test-label.dat
Saving train-data.dat to train-data.dat
Saving train-label.dat to train-label.dat


# Import necassary libraries

In [21]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import zero_one_loss, classification_report, label_ranking_average_precision_score, mean_squared_error, mean_absolute_error, r2_score,coverage_error
from sklearn.preprocessing import MultiLabelBinarizer
import warnings
warnings.filterwarnings("ignore")

# Read the contents of train-data.dat and test-data.dat


In [14]:
with open('train-data.dat', 'r') as train_file:
    train_data = train_file.readlines()

with open('test-data.dat', 'r') as test_file:
    test_data = test_file.readlines()

import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Preprocess the data

In [16]:
stemmer = PorterStemmer()

def preprocess_document(document):
    # Remove special characters and convert to lowercase (lowercase also not needed, just in case)
    document = re.sub(r'[^a-zA-Z0-9\s<>]', '', document.lower())

    # Split into sentences and words
    sentences = document.split('\n')
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

    # Remove words and numbers enclosed in < and > symbols
    preprocessed_sentences = []
    for sentence in sentences:
        words = sentence.split()
        words = [word for word in words if not (word.startswith('<') and word.endswith('>'))]
        preprocessed_sentences.append(' '.join(words))

    # Wrap each line in single quotes
    preprocessed_document = "'" + "'\n'".join(preprocessed_sentences) + "'"

    return preprocessed_document

preprocessed_train_data = [preprocess_document(document) for document in train_data]
preprocessed_test_data = [preprocess_document(document) for document in test_data]

# Print the first 5 lines of preprocessed train-data
print("Preprocessed train-data:")
for i in range(5):
    print(preprocessed_train_data[i])
    print()

# Print the first 5 lines of preprocessed test-data
print("Preprocessed test-data:")
for i in range(5):
    print(preprocessed_test_data[i])
    print()


Preprocessed train-data:
'6705 5997 8310 3606 674 8058 5044 4836 4312 5154 8310 4225'

'1827 1037 8482 483 3567 6172 6172 2892 1362 787 399 777 1332 318 769 4621 3199 1480 6213 971 6890 5909 15 3445 2475 324 4138 3404 6176 65 2926 1375 7705 709 1323 1652 5735 7439 3445 2475 4396 4382 1798 1870 1845 4410 7758 4278 7775 318 769 2898 1923 8193 2930 485 5951 6123 2480 3158 2193 2438 7485 3412 5304 5644 3734 7459 1652 3174 8343 426 1968 7506 561 4051 2722 6717 6172 3432 7859 4595 2351 5951 135 7023 7215 7038 7221 560 4631 4411 1348 8185 5192 3445 2475 5696 5154 361 7953 4245 560 2351 4410 7758 2174 1120 6688 7705 2747 1296 1652 134 5909 7953 4245'

'8482 787 2169 4077 6335 1137 1598 1055 1223 2735 3492 413 838 4457 8392 3631 3699 42 5217 4281 6309 2912 6629 1985 2781 2155 5217 8032 860 7998 1469 4077 682 5950 7654 1561 5217 137 4032 42 5217 29 365 5339 4081 4386 7654 679 5217 620 8197 679 8403 42 5217 2134 5427 980 550 316 762 5582 3148 6039 5217 7378 470 674 8197 1072 42 5217 4164 470 5208

# Vectorize the data

In [17]:
vectorizer = CountVectorizer()
train_vectors = vectorizer.fit_transform(preprocessed_train_data)
test_vectors = vectorizer.transform(preprocessed_test_data)

# Access the vocabulary and its corresponding indices

In [18]:
vocabulary = vectorizer.vocabulary_
feature_names = vectorizer.get_feature_names_out()

# Print the vocabulary
print("Vocabulary:")
print(vocabulary)
print()

Vocabulary:
{'6705': 6334, '5997': 5547, '8310': 8116, '3606': 2894, '674': 6372, '8058': 7835, '5044': 4490, '4836': 4259, '4312': 3678, '5154': 4612, '4225': 3581, '1827': 919, '1037': 42, '8482': 8306, '483': 4252, '3567': 2850, '6172': 5742, '2892': 2101, '1362': 403, '787': 7626, '399': 3319, '777': 7515, '1332': 370, '318': 2420, '769': 7426, '4621': 4021, '3199': 2441, '1480': 534, '6213': 5788, '971': 8479, '6890': 6539, '5909': 5450, '15': 555, '3445': 2715, '2475': 1638, '324': 2487, '4138': 3484, '3404': 2670, '6176': 5746, '65': 6105, '2926': 2139, '1375': 417, '7705': 7444, '709': 6760, '1323': 360, '1652': 725, '5735': 5257, '7439': 7148, '4396': 3770, '4382': 3755, '1798': 886, '1870': 967, '1845': 939, '4410': 3787, '7758': 7502, '4278': 3639, '7775': 7521, '2898': 2107, '1923': 1026, '8193': 7985, '2930': 2144, '485': 4274, '5951': 5497, '6123': 5688, '2480': 1644, '3158': 2396, '2193': 1325, '2438': 1597, '7485': 7199, '3412': 2679, '5304': 4779, '5644': 5156, '3734':

In [19]:
# Load training and test labels
y_train = pd.read_csv('/content/train-label.dat', sep='\t', header=None)[0].values
y_test = pd.read_csv('/content/test-label.dat', sep='\t', header=None)[0].values

# Convert the label strings into binary matrix representation
y_train_bin = np.array([list(map(int, label.split())) for label in y_train])
y_test_bin = np.array([list(map(int, label.split())) for label in y_test])

# Print the number of labels
print("Number of labels:")
print("Train labels:", y_train_bin.shape[1])
print("Test labels:", y_test_bin.shape[1])
print()

Number of labels:
Train labels: 20
Test labels: 20



# Train and evaluate models

In [22]:
# Define classifiers
classifiers = [
    ("Linear SVM", OneVsRestClassifier(LinearSVC(max_iter=10000))),
    ("Logistic Regression", ClassifierChain(LogisticRegression(max_iter=10000))),
]

# Create an empty list to store the predicted labels for each classifier
y_pred_list = []

# Iterate over classifiers
for classifier_name, classifier in classifiers:
    # Train and evaluate the classifier
    classifier.fit(train_vectors, y_train_bin)

    # Predict labels for the test set
    y_pred_bin = classifier.predict(test_vectors)

    # Print the classification results
    print(classifier_name)
    report = classification_report(y_test_bin, y_pred_bin, zero_division=0, output_dict=True)
    # Print the classification report
    print("Classification Report:")
    df_report = pd.DataFrame(report).transpose()
    print(df_report)
    print()

    # Calculate additional metrics
    ranking_loss = 1 - label_ranking_average_precision_score(y_test_bin, y_pred_bin)
    avg_precision = label_ranking_average_precision_score(y_test_bin, y_pred_bin)

    print(f"Ranking Loss: {ranking_loss:.2f}")
    print(f"Average Precision: {avg_precision:.2f}")
    print()

    # Calculate regression metrics
    mse = mean_squared_error(y_test_bin, y_pred_bin)
    mae = mean_absolute_error(y_test_bin, y_pred_bin)
    r2 = r2_score(y_test_bin, y_pred_bin)

    # Calculate subset accuracy
    
    #subset_acc = accuracy_score(y_test_bin, y_pred_bin)
    print('subset accuracy   : %.2f' % (1-zero_one_loss(y_test_bin, y_pred_bin)))
    # Calculate coverage error
    coverage_err = coverage_error(y_test_bin, y_pred_bin)  # Calculate coverage error


    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"R-squared (R2): {r2:.2f}")
    print()


Linear SVM
Classification Report:
              precision    recall  f1-score  support
0              0.603922  0.630502  0.616925    977.0
1              0.375940  0.438596  0.404858    228.0
2              0.511041  0.519897  0.515431   1558.0
3              0.539474  0.551075  0.545213    372.0
4              0.462230  0.489524  0.475486   1050.0
5              0.274278  0.389199  0.321786    537.0
6              0.341772  0.346154  0.343949    702.0
7              0.458621  0.493049  0.475212   1079.0
8              0.428740  0.453300  0.440678    803.0
9              0.385900  0.430642  0.407045    483.0
10             0.353796  0.431953  0.388988    507.0
11             0.348425  0.370293  0.359026    478.0
12             0.326316  0.365422  0.344764    509.0
13             0.331787  0.402817  0.363868    355.0
14             0.419903  0.441327  0.430348    392.0
15             0.356828  0.367347  0.362011    441.0
16             0.324590  0.368030  0.344948    269.0
17          

# Conclusions

**Linear SVM:**

Classification Report: The precision, recall, and F1-score values vary across different classes. The weighted average F1-score is 0.441, which indicates a reasonable overall performance.
Ranking Loss: 0.61, indicating moderate ranking quality.
Average Precision: 0.39, suggesting that the classifier has room for improvement in precision.
Subset Accuracy was very bad in this case.
(MSE: 0.17, indicating a moderate average squared difference between predicted and true values.
MAE: 0.17, suggesting a moderate average absolute difference.
R2: -0.42, which implies that the model does not explain the variance well.)

**Logistic Regression:**

Classification Report: . The weighted average F1-score is 0.464, indicating a slightly better overall performance compared to Linear SVM.
Ranking Loss: 0.58, suggesting a slightly better ranking quality compared to Linear SVM.
Average Precision: 0.42, showing a slight improvement in precision compared to Linear SVM.
Subset Accuracy: slightly better than Linear SVM, but again, not very good.
MSE: 0.14, indicating a lower average squared difference compared to Linear SVM.
MAE: 0.14, showing a slightly lower average absolute difference compared to Linear SVM.
R2: -0.18, indicating that the model explains the variance slightly better than Linear SVM.

Based on these metrics, the Logistic Regression classifier performs slightly better than the Linear SVM classifier. It has slightly higher F1-scores, better ranking quality, and a slightly higher subset accuracy. Additionally, it achieves a lower MSE and MAE, suggesting better performance in terms of regression metrics. However, both models have relatively low subset accuracy and R2 values, indicating room for improvement.